In [ ]:
import json
import urllib.parse
import urllib.request
import requests

In [ ]:
MAPQUEST_API_KEY = "mIefa9gls1dRutE576TfkQxSocKhGmEx"
BASE_MAPQUEST_URL = "http://www.mapquestapi.com/"

# function to build url with positions
def build_search_url_pos(start_posi, end_posi) -> str:
    query_parameters = [('key',MAPQUEST_API_KEY),('from',start_posi),('to',end_posi)]
    return BASE_MAPQUEST_URL +"directions/v2/route?" + urllib.parse.urlencode(query_parameters)

# function to build url with lat&long
def build_search_url_latlong(start_lat:str, start_lon:str, 
                     ends_lat:str, ends_lon:str) -> str:
    query_parameters = [('key',MAPQUEST_API_KEY),('from',start_lat+','+start_lon),('to',ends_lat+','+ends_lon)]
    return BASE_MAPQUEST_URL +"directions/v2/route?" + urllib.parse.urlencode(query_parameters)



# function to build url with start position and end lat long
def build_search_url_pos_latlong(start_posi, 
                       ends_lat:str, ends_lon:str) -> str:
    query_parameters = [('key',MAPQUEST_API_KEY),('from',start_posi),('to',ends_lat+','+ends_lon)]
    return BASE_MAPQUEST_URL +"directions/v2/route?" + urllib.parse.urlencode(query_parameters)

# function that get the web result(dictionary) of MAPQUEST
def get_result(url: str) -> dict:
    result = None
    try:
        result = urllib.request.urlopen(url)
        json_text = result.read().decode(encoding = 'utf-8')
#        print(json_text)
        return json.loads(json_text)

    finally:
        if result != None:
            result.close()
            
# get distance, units in miles
def get_route_distance(json_result):
    dis = json_result['route']['distance']
    return dis

#Start location is Exit 1A of I-5; end location is I-5 Exit 108

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Modeling
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Modeling
Data  Geo.ipynb  remoteDataList


In [ ]:
exits = pd.read_csv('Data/I5Exits.csv')
exits

,attributes.ExitNumber,geometry.x,geometry.y
0,1 A,-122.672895,45.621707
1,1 C,-122.667187,45.627694
2,2,-122.661702,45.644626
3,3,-122.663286,45.654278
4,4,-122.664597,45.671567
...,...,...,...
726,NaN,-122.429281,47.234529
727,NaN,-122.432021,47.236890
728,NaN,-122.205107,47.470999
729,NaN,-122.454182,47.230811


In [ ]:
# End 108, Start 1A
end_lat = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.y']
end_long = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.x']
start_lat = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.y']
start_long = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.x']

In [ ]:
url = build_search_url_latlong(str(start_lat), str(start_long), str(end_lat), str(end_long))
result = get_result(url)
get_route_distance(result)

107.686

# Filter gas stations within 5 mi range of I-5 1A to 108 Corridor

In [ ]:
sessionId = result['route']['sessionId']

In [ ]:
def build_corridor_search_url() -> str:
    query_parameters = [('key', MAPQUEST_API_KEY)]
    return BASE_MAPQUEST_URL +"search/v2/corridor?" + urllib.parse.urlencode(query_parameters)

def build_corridor_search_body(dataList) -> str:
    json_map = {
        "sessionId": sessionId,
        "remoteDataList": dataList,
        "options": {
            "width": '5',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)

def post_corridor_search_request(dataList):
    result = None
    url = build_corridor_search_url()
    body = build_corridor_search_body(dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()

def getRemoteDataList(df,lt,lg,name):
  df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
  remoteDataList = []
  for row in range(df.shape[0]):
    inner = {}
    r_data = df.iloc[row]
    inner['key'] = str(row)
    inner['name'] = r_data[name]
    inner['shapePoints'] = r_data['shapePoints']
    remoteDataList.append(inner)
  return remoteDataList


In [ ]:
gas_station = pd.read_csv('Data/wa_gas_stations.csv')
remoteDataList = getRemoteDataList(gas_station,'Latitude','Longitude','name')

result = post_corridor_search_request(remoteDataList)

In [ ]:
print(result['resultsCount'])

259


In [ ]:
filtered_gas_stations = pd.DataFrame(result['searchResults'])

In [ ]:
filtered_gas_stations.iloc[258]

distanceUnit                             m
distance                            0.0852
name                                 Shell
sourceName                      remoteData
resultNumber                           259
key                                   1927
shapePoints     [45.6578361, -122.5767784]
Name: 258, dtype: object

# For each gas station, find closest exit

In [ ]:

def build_radius_search_url() -> str:
    query_parameters = [('key', MAPQUEST_API_KEY)]
    return BASE_MAPQUEST_URL +"search/v2/radius?" + urllib.parse.urlencode(query_parameters)

def build_radius_search_body(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '5',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)

def post_radius_search_request(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()


In [ ]:
exits = pd.read_csv('Data/I5Exits.csv')
exitDataList = getRemoteDataList(exits,'geometry.y','geometry.x','attributes.ExitNumber')
origin = filtered_gas_stations.iloc[0]['shapePoints']
exit_result = post_radius_search_request(origin, exitDataList)
exit_result['resultsCount']

[{'key': '0', 'name': '1 A', 'shapePoints': [45.6217068871334, -122.67289510988658]}, {'key': '1', 'name': '1 C', 'shapePoints': [45.627693986717325, -122.6671873195742]}, {'key': '2', 'name': '2', 'shapePoints': [45.64462647537196, -122.66170209636194]}, {'key': '3', 'name': '3', 'shapePoints': [45.65427776692005, -122.66328588817734]}, {'key': '4', 'name': '4', 'shapePoints': [45.67156705568538, -122.66459709933368]}, {'key': '5', 'name': '5', 'shapePoints': [45.68768258454696, -122.66385789606024]}, {'key': '6', 'name': '7 A', 'shapePoints': [45.71263847516081, -122.65287710107106]}, {'key': '7', 'name': '7 B', 'shapePoints': [45.71730024172571, -122.65454942746192]}, {'key': '8', 'name': '9', 'shapePoints': [45.74559686551165, -122.66100567206163]}, {'key': '9', 'name': '11', 'shapePoints': [45.76317025998826, -122.66644606162782]}, {'key': '10', 'name': '14', 'shapePoints': [45.80636153627611, -122.68133336358036]}, {'key': '11', 'name': '16', 'shapePoints': [45.84260918071932, -1

6

In [ ]:
def get_closest_exit(origin):
    exit_result = post_radius_search_request(origin, exitDataList)
    filtered_exits = pd.DataFrame(exit_result['searchResults'])
    closest_exit = filtered_exits.iloc[filtered_exits['distance'].argmin()]
    return closest_exit['distance'], closest_exit['name'], closest_exit['shapePoints'] 

In [ ]:
dist_to_exit = []
for gas_r in range(filtered_gas_stations.shape[0]):
  gas_data = filtered_gas_stations.iloc[gas_r]
  dist, exit_n, exit_p = get_closest_exit(gas_data['shapePoints'])
  dist_to_exit.append([gas_data['key'],gas_data['shapePoints'][0],gas_data['shapePoints'][1],exit_n,exit_p[0],exit_p[1],dist])
dist_to_exit = pd.DataFrame(dist_to_exit)
dist_to_exit.columns = ['gas_key','gas_lat','gas_long','exit_name','exit_lat','exit_long','distance']

In [ ]:
dist_to_exit.to_csv('Data/gas_station_w_dist_to_exit_I5.csv')